In [138]:
from sklearn.datasets import load_breast_cancer
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sympy.polys.subresultants_qq_zz import backward_eye


In [139]:
import numpy as np

dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_tensor = torch.tensor(X_train,dtype=torch.float32)
X_test_tensor = torch.tensor(X_test,dtype=torch.float32)
y_train_tensor = torch.tensor(y_train,dtype=torch.float32)
y_test_tensor = torch.tensor(y_test,dtype=torch.float32)

C:\Users\mailt\AppData\Local\Temp\ipykernel_14688\3801309834.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train,dtype=torch.float32)
C:\Users\mailt\AppData\Local\Temp\ipykernel_14688\3801309834.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test,dtype=torch.float32)


In [140]:
type(X_train)

numpy.ndarray

In [141]:
from torch.utils.data import Dataset,DataLoader

In [142]:
import torch.nn as nn

class CustomDataset(Dataset):
    def __init__(self,features,label):
        self.features = features
        self.label = label
    def __len__(self):
        return len(self.label)
    def __getitem__(self, idx):
        return self.features[idx], self.label[idx]



In [143]:
train_dataset = CustomDataset(X_train_tensor,y_train_tensor)
test_dataset = CustomDataset(X_test_tensor,y_test_tensor)
print(train_dataset.__len__())
print(test_dataset.__len__())

455
114


In [144]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [145]:
loss_function = nn.BCELoss()

In [146]:
class Myneuralnetwork(nn.Module):
    def __init__(self,num_feature):
        super().__init__()
        self.linear1 = nn.Linear(num_feature,10)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(10,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,features):
        out = self.linear1(features)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out




In [147]:
learning_rate = 0.01
epochs = 200


In [148]:
model = Myneuralnetwork(X_train.shape[1])

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [149]:
for epoch in range(epochs):
    for batch_features, batch_label in train_loader:
        y_pred = model(batch_features)
        loss = loss_function(y_pred, batch_label.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch: {epoch+1}, loss: {loss}")


epoch: 1, loss: 0.6994326710700989
epoch: 2, loss: 0.7262479066848755
epoch: 3, loss: 0.7087395787239075
epoch: 4, loss: 0.6993163824081421
epoch: 5, loss: 0.671747624874115
epoch: 6, loss: 0.6036185026168823
epoch: 7, loss: 0.6371415853500366
epoch: 8, loss: 0.639208972454071
epoch: 9, loss: 0.6302467584609985
epoch: 10, loss: 0.530420184135437
epoch: 11, loss: 0.4963884651660919
epoch: 12, loss: 0.5003877878189087
epoch: 13, loss: 0.5314141511917114
epoch: 14, loss: 0.3598819375038147
epoch: 15, loss: 0.3271043598651886
epoch: 16, loss: 0.25161880254745483
epoch: 17, loss: 0.2218659222126007
epoch: 18, loss: 0.3249233365058899
epoch: 19, loss: 0.3840339481830597
epoch: 20, loss: 0.37220191955566406
epoch: 21, loss: 0.22718404233455658
epoch: 22, loss: 0.2869856357574463
epoch: 23, loss: 0.3472844660282135
epoch: 24, loss: 0.15995056927204132
epoch: 25, loss: 0.20743976533412933
epoch: 26, loss: 0.22448907792568207
epoch: 27, loss: 0.10102248191833496
epoch: 28, loss: 0.15967932343482

In [150]:
model.eval()
accuracy_list = []
with torch.no_grad():
    for batch_features, batch_label in test_loader:
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.5).float()

    batch_accuracy = (y_pred.view(-1) == batch_label).float().mean().item()
    accuracy_list.append(batch_accuracy)

overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f"overall accuracy: {overall_accuracy}")



overall accuracy: 1.0
